In [0]:
# Widget to take user input (any case)
dbutils.widgets.text("client_name", "", "Client Name")
client_name = dbutils.widgets.get("client_name") 

In [0]:
#CDAP Connection
sql_server_name = dbutils.secrets.get(scope="ETL-KeyVault", key="sql-etl-cdap-server")
sql_server_database = dbutils.secrets.get(scope="ETL-KeyVault", key="sql-etl-cdap-db")
sql_server_user_name = dbutils.secrets.get(scope="ETL-KeyVault", key="sql-etl-cdap-user")
sql_server_password = dbutils.secrets.get(scope="ETL-KeyVault", key="sql-etl-cdap-pwd")

jdbc_url = (f"jdbc:sqlserver://{sql_server_name}.database.windows.net:1433;"
                f"database={sql_server_database};"
                f"user={sql_server_user_name}@{sql_server_name};"
                f"password={sql_server_password};"
                f"encrypt=true;trustServerCertificate=false;"
                f"hostNameInCertificate=*.database.windows.net;loginTimeout=30;")



In [0]:
# Get CDAP ClientName
query_clientname = f"""
                        select * from [CDAP].[Client] where Name like '%{client_name}%'
                        """

cdap_client_result_df = spark.read.format("jdbc") \
                .option("url", jdbc_url) \
                .option("query", query_clientname) \
                .option("user", sql_server_user_name) \
                .option("password", sql_server_password) \
                .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
                .load()


# Convert the DataFrame to a list of dictionaries 'Name' column and collect as Python Row objects
rows_client_name = cdap_client_result_df.select("DAPClientName").collect()

# Convert each Row to a dictionary
name_dict_list = [row.asDict() for row in rows_client_name]
# Save the cdap client name
cdap_client_name = name_dict_list
if name_dict_list:
    name_value = name_dict_list[0]['DAPClientName']
    print(name_value)
else:
    print("No name found")


State_of_NorthCarolina_Aetna


In [0]:
#Get Plan Sponsor Name

query_plansponsorname = f""" select c.Name,ClientCode,ps.PlanSponsorName from [CDAP].[Client] c
inner join [CDAP].[ClientSponsor] cs on c.ClientKey=cs.ClientKey
inner join [CDAP].[PlanSponsor] ps on cs.PlanSponsorId=ps.PlanSponsorId
where c.Name like '%{client_name}%' """
display(query_plansponsorname)

df_plansponsorname = spark.read.format("jdbc") \
                .option("url", jdbc_url) \
                .option("query", query_plansponsorname) \
                .option("user", sql_server_user_name) \
                .option("password", sql_server_password) \
                .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
                .load()

display(df_plansponsorname)
rows_plansponsor_name = [row['PlanSponsorName'] for row in df_plansponsorname.select("PlanSponsorName").collect()]

# Convert each Row to a dictionary
plansponsor_dict_list = [{'PlanSponsorName': row} for row in rows_plansponsor_name]
# Save the cdap plansponsor name
cdap_plansponsor_name = plansponsor_dict_list
if plansponsor_dict_list:
    name_ps_value = cdap_plansponsor_name[0]['PlanSponsorName']
    print(name_ps_value)
else:
    print("No name found")



" select c.Name,ClientCode,ps.PlanSponsorName from [CDAP].[Client] c\ninner join [CDAP].[ClientSponsor] cs on c.ClientKey=cs.ClientKey\ninner join [CDAP].[PlanSponsor] ps on cs.PlanSponsorId=ps.PlanSponsorId\nwhere c.Name like '%State_of_NorthCarolina_Aetna%' "

Name,ClientCode,PlanSponsorName
State_of_NorthCarolina_Aetna,State_of_NorthCarolina_Aetna,Aetna


Aetna


In [0]:
user_input = "name_value"

query_dx_client = f"""
SELECT Name as ClientId
FROM etl_dev.dev_structured.lntrninternal_carehub_client_unified t
WHERE lower(LEFT(t.name, CHARINDEX('_', concat(t.name, '_')) - 1)) = lower(LEFT('{user_input}', CHARINDEX('_', concat('{user_input}', '_')) - 1))
""" 
spark.sql(query_dx_client).display()


ClientId


In [0]:
# Prep zone metrics validation
query = f"""
SELECT file_name, client, input_records_count, new_records_count, updated_records_count, excluded_records_count, run_datetime
FROM etl_dev.dev_prep.plansponsorclaims_metrics
WHERE client = '{name_value}' ORDER BY run_datetime DESC LIMIT 10
"""
df_plansponsormetrics = spark.sql(query)
display(df_plansponsormetrics)


file_name,client,input_records_count,new_records_count,updated_records_count,excluded_records_count,run_datetime
20290108.Aetna.Claims.Weekly.csv,State_of_NorthCarolina_Aetna,16,9,0,7,2025-12-15T03:08:42.755Z
20280116.Aetna.Claims.Weekly.txt,State_of_NorthCarolina_Aetna,15,11,0,4,2025-12-08T15:23:31.891Z
20280115.Aetna.Claims.Weekly.txt,State_of_NorthCarolina_Aetna,15,11,0,4,2025-12-05T19:47:33.416Z
20280114.Aetna.Claims.Weekly.txt,State_of_NorthCarolina_Aetna,15,12,0,3,2025-12-05T19:24:28.090Z
20280107.Aetna.Claims.Weekly.txt,State_of_NorthCarolina_Aetna,84579,84579,0,0,2025-12-03T16:44:11.168Z
20280106.Aetna.Claims.Weekly.txt,State_of_NorthCarolina_Aetna,2,2,0,0,2025-12-03T14:38:43.164Z
20280104.Aetna.Claims.Weekly.txt,State_of_NorthCarolina_Aetna,2,2,0,0,2025-12-02T15:52:07.815Z
20280102.Aetna.Claims.Weekly.txt,State_of_NorthCarolina_Aetna,2,2,0,0,2025-12-02T14:18:12.619Z
20280101.Aetna.Claims.Weekly.txt,State_of_NorthCarolina_Aetna,2,2,0,0,2025-12-02T14:13:21.380Z
2027111.Aetna.Claims.Weekly.txt,State_of_NorthCarolina_Aetna,2,2,0,0,2025-12-02T03:19:24.476Z


In [0]:
# Prep-zone data layout and validation
query_Medical = f"""
select * from etl_dev.dev_prep.plansponsorclaims_medical_{name_value} order by created_timestamp desc limit 15
"""
df_plansponsorclaims_medical = spark.sql(query_Medical)

# Display the DataFrame
display(df_plansponsorclaims_medical)

# Get the file date
query_filedate = f"""
select file_date from etl_dev.dev_prep.plansponsorclaims_medical_{name_value} order by created_timestamp desc limit 1
"""
df_filedate = spark.sql(query_filedate)
file_date = df_filedate.select("file_date").first()["file_date"]
display(file_date)

hierarchy_level_3,hierarchy_level_5,hierarchy_level_6,benefit_identification_code,creation_date,employee_s_ssn_number,non_ee_ssn,employee_first_name,employee_last_name,employee_street,employee_city,employee_state,employee_zip,patient_account_number,member_s_dob,member_s_relationship_to_employee,patient_first_name,patient_last_name,date_service_started,date_service_stopped,date_processed,source_claim_number,expense_line_number,adjustment_indicator,original_claim_number,claim_category,place_of_service,covered_amount,copay_amount,deductible_amount,coinsurance_amount,benefits_payable_amount,total_paid_amount,pay_to_provider_zip,cpt4_procedure_code,icd_primary_diagnosis_code,dx_code_2,dx_code_3,dx_code_4,dx_code_5,dx_code_6,dx_code_7,dx_code_8,dx_code_9,dx_code_10,dx_code_11,dx_code_12,upid,insured_dob,insured_gender,patient_gender,patient_address,patient_address_2,patient_city,patient_zip,patient_phone,file_date,created_timestamp,modified_timestamp,claim_id,validation_results
192685,10,47,46501,null,218330296,289624172,ANIKA,PAYNE,4001 TARRANT TRACE CIRCLE,HIGH POINT,NC,27265,250193,09/01/2025,D,WILHELMINA,PAYNE,09/12/2025,09/12/2025,null,E0FDPN1BM00,2,,,M,11,122.7,0,0,0,122.7,122.7,27262,17250,L980,Z2911,P0260,Z2911,Z2911,,,,,,P929,P929,null,null,null,M,null,null,null,null,123-333-3342,20290108,2025-12-15T03:07:35.458Z,2025-12-15T03:07:35.458Z,E0FDPN1BM00,null
192681,11,24,46501,null,238659619,289904110,JEREMY,CHASE,380 FOREHAND ROAD,PIKEVILLE,NC,27863,248227,09/02/2025,S,WYATT,CHASE,09/08/2025,09/08/2025,null,E4PDPQWWQ00,1,,,M,11,115.11,0,0,0,115.11,115.11,27609,99213,H04531,P0510,Z2911,Z2911,Z2911,,,,,,P0510,P0510,null,null,null,M,null,null,null,null,123-333-3339,20290108,2025-12-15T03:07:35.458Z,2025-12-15T03:07:35.458Z,E4PDPQWWQ00,null
192685,11,30,46501,null,246154843,290142942,LIAUNA,JORDAN,855 MARY JORDAN LANE,FAYETTEVILLE,NC,28311,210366,01/14/2001,D,JASMIN,JORDAN,06/03/2025,06/03/2025,null,EZJNL932502,1,A,EZJNL932501,M,11,0,0,0,0,0,0,28304,99213,Z3041,E6601,Z2911,Z2911,Z2911,,,,,,E6601,E6601,null,null,null,M,null,null,null,null,123-333-3336,20290108,2025-12-15T03:07:35.458Z,2025-12-15T03:07:35.458Z,EZJNL932501EZJNL932502,null
192681,11,24,46501,null,238659619,289904110,JEREMY,CHASE,380 FOREHAND ROAD,PIKEVILLE,NC,27863,249248,09/02/2025,S,WYATT,CHASE,09/19/2025,09/09/2025,null,E8FDPQYH400,1,,,M,11,115.11,0,0,0,115.11,115.11,27609,99213,P0510,Z2911,Z2911,Z2911,Z2911,,,,,,,,null,null,null,M,null,null,null,null,123-333-3340,20290108,2025-12-15T03:07:35.458Z,2025-12-15T03:07:35.458Z,E8FDPQYH400,null
192681,11,24,46501,null,238659619,289904110,JEREMY,CHASE,380 FOREHAND ROAD,PIKEVILLE,NC,27863,248093,09/02/2025,S,WYATT,CHASE,09/05/2025,09/05/2025,null,E3TYPMRYX00,1,,,M,11,137.58,0,0,0,137.58,137.58,27609,99381,Z00110,P0510,Z2911,Z2911,Z2911,,,,,,P0510,P0510,null,null,null,M,null,null,null,null,123-333-3338,20290108,2025-12-15T03:07:35.458Z,2025-12-15T03:07:35.458Z,E3TYPMRYX00,null
192681,11,24,46501,null,238659619,289904110,JEREMY,CHASE,380 FOREHAND ROAD,PIKEVILLE,NC,27863,250476,09/02/2025,S,WYATT,CHASE,10/03/2025,10/03/2025,null,E737PL6HF00,4,,,M,11,3.67,0,0,0,3.67,3.67,27609,96160,Z00129,P0510,Z2911,Z2911,Z2911,,,,,,P0510,P0510,null,null,null,M,null,null,null,null,123-333-3344,20290108,2025-12-15T03:07:35.458Z,2025-12-15T03:07:35.458Z,E737PL6HF00,null
192685,10,47,46501,null,218330296,289624172,ANIKA,PAYNE,4001 TARRANT TRACE CIRCLE,HIGH POINT,NC,27265,250056,09/01/2025,D,WILHELMINA,PAYNE,09/09/2025,09/09/2025,null,E4FDPNZC100,1,,,M,11,135.42,25,0,0,110.42,110.42,27262,99213,P929,Z2911,Z2911,Z2911,Z2911,,,,,,,,null,null,null,M,null,null,null,null,123-333-3341,20290108,2025-12-15T03:07:35.458Z,2025-12-15T03:07:35.458Z,E4FDPNZC100,null
192685,10,48,46501,null,240515919,289961305,DONETTA,CHERRY,1018 HUNTER ST,ELIZABETH CITY,NC,27909,357135,09/12/2025,S,KAIRO,CHERRY,09/24/2025,09/24/2025,null,ESPDPRCG200,2,,,M,11,5.25,0,0,0,5.25,5.25,27895,82962,Z00111,R258,Z09,Z2911,Z2911,,,,,,R258,R258,null,null,null,M,null,null,null,null,123-2

'20290108'

In [0]:
#Structure zone validation

query_standard = f"""
select * from etl_dev.dev_structured.plansponsorclaims_medical_standard where 
file_effective_date= '{file_date}'
"""
# need to include clientid
 #and clientid IN (
  #  SELECT ClientCode
   # FROM etl_dev.dev_structured.lntrninternal_carehub_client_unified
    #WHERE LOWER(name) LIKE LOWER('%{client_name}%')
#)
print()
display(query_standard)
df_standard = spark.sql(query_standard)
display(df_standard)


"\nselect * from etl_dev.dev_structured.plansponsorclaims_medical_standard where \nfile_effective_date= '20251207'\n"

file_effective_date,claim_id,claim_type,claim_tob,insurance_plan_name,plan_sponsor_name,insured_id,insured_group_number,insured_first_name,insured_last_name,insured_middle_initial,insured_ssn,insured_dob,insured_sex,insured_address,insured_address2,insured_city,insured_state,insured_zip,insured_phone,insured_email,patient_first_name,patient_last_name,patient_middle_initial,patient_ssn,patient_dob,patient_sex,patient_address,patient_address2,patient_city,patient_state,patient_zip,patient_phone,patient_email,patient_relationship_code,patient_relationship_desc,icd_version,dx_code_1,dx_code_2,dx_code_3,dx_code_4,dx_code_5,dx_code_6,dx_code_7,dx_code_8,dx_code_9,dx_code_10,dx_code_11,dx_code_12,drg_code,begin_date,end_date,pos_code,procedure_code,revenue_code,revenue_code_description,ndc,mod_1,rendering_provider_name,rendering_provider_npi,facility_name,facility_npi,facility_zip,claim_line,member_sequence_number,client_name,created_timestamp,modified_timestamp,clientId
20251207,FJ50699240311,null,null,null,UNITED HEALTHCARE,null,null,Aalen,Adham,null,null,0066-01-01,M,453 LAKE TRAIL,null,WINSTED,MN,55395,null,null,Alfred,John,null,null,1975-09-20,M,453 LAKE TRAIL,null,WINSTED,MN,55395,null,null,1,Self,null,E6609,null,null,null,null,null,null,null,null,null,null,null,null,2025-10-21,2025-10-21,null,90837,null,null,null,null,ADAM SARFF,1609261106,null,null,null,1,null,HNTB,2025-12-10 14:57:43.475,2025-12-10 14:57:43.475,HNTB
20251207,FJ26178875974,null,null,null,UNITED HEALTHCARE,null,null,Neli,Garcia Roman,null,null,0069-03-15,M,N7031 537TH ST,null,MENOMONIE,WI,54751,null,null,Aldo,Spada,null,null,1969-03-11,M,N7031 537TH ST,null,MENOMONIE,WI,54751,null,null,1,Self,null,G500,null,null,null,null,null,null,null,null,null,null,null,null,2025-10-14,2025-10-14,null,85025,null,null,null,null,KIRSTEN JOY PIERCE,1871938142,null,null,null,4,null,HNTB,2025-12-10 14:57:43.475,2025-12-10 14:57:43.475,HNTB


In [0]:
#curation zone validation - demographic match
query_demomatch = f""" select MemberId,patient_first_name,patient_last_name,patient_dob,patient_sex,patient_relationship_code,dx_code_1,dx_code_2,dx_code_3,dx_code_4,dx_code_5,demographic_match,demographic_match_tier from etl_dev.dev_curation.plansponsorclaims_medical_demographicmatch where file_effective_date='{file_date}' and plan_sponsor_name = '{name_ps_value}'"""
display(query_demomatch)
df_demomatch = spark.sql(query_demomatch)
display(df_demomatch)

member_ids = [row["MemberId"] for row in df_demomatch.select("MemberId").collect()]
print(member_ids)


" select MemberId,patient_first_name,patient_last_name,patient_dob,patient_sex,patient_relationship_code,dx_code_1,dx_code_2,dx_code_3,dx_code_4,dx_code_5,demographic_match,demographic_match_tier from etl_dev.dev_curation.plansponsorclaims_medical_demographicmatch where file_effective_date='20251207' and plan_sponsor_name = 'UNITED HEALTHCARE'"

MemberId,patient_first_name,patient_last_name,patient_dob,patient_sex,patient_relationship_code,dx_code_1,dx_code_2,dx_code_3,dx_code_4,dx_code_5,demographic_match,demographic_match_tier
B12DBB3E-3187-ED11-9D7A-000D3A3F8513,Alfred,John,1975-09-20,M,1,E6609,null,null,null,null,true,tier1
972DBB3E-3187-ED11-9D7A-000D3A3F8513,Aldo,Spada,1969-03-11,M,1,G500,null,null,null,null,true,tier1


['B12DBB3E-3187-ED11-9D7A-000D3A3F8513', '972DBB3E-3187-ED11-9D7A-000D3A3F8513']


In [0]:

member_ids_str = ",".join([f"'{mid}'" for mid in member_ids])

if not member_ids:
    print("No member IDs found; skipping query.")
else:
    query_tagged = f"""
    select MemberId, patient_first_name, patient_last_name, patient_dob, patient_sex, patient_relationship_code, tag_msk, tag_msk_date, tag_msk_code, tag_bar, tag_bar_date, tag_bar_code, created_timestamp 
    from etl_dev.dev_curation.plansponsorclaims_medical_tagged
    where MemberId in ({member_ids_str})
    """
    display(query_tagged)
    df_tagged = spark.sql(query_tagged)
    display(df_tagged)


"\n    select MemberId, patient_first_name, patient_last_name, patient_dob, patient_sex, patient_relationship_code, tag_msk, tag_msk_date, tag_msk_code, tag_bar, tag_bar_date, tag_bar_code, created_timestamp \n    from etl_dev.dev_curation.plansponsorclaims_medical_tagged\n    where MemberId in ('B12DBB3E-3187-ED11-9D7A-000D3A3F8513','972DBB3E-3187-ED11-9D7A-000D3A3F8513')\n    "

MemberId,patient_first_name,patient_last_name,patient_dob,patient_sex,patient_relationship_code,tag_msk,tag_msk_date,tag_msk_code,tag_bar,tag_bar_date,tag_bar_code,created_timestamp
972DBB3E-3187-ED11-9D7A-000D3A3F8513,Aldo,Spada,1969-03-11,M,1,true,2025-10-14,G500,false,null,null,2025-12-10T15:04:28.710Z
B12DBB3E-3187-ED11-9D7A-000D3A3F8513,Alfred,John,1975-09-20,M,1,false,null,null,true,2025-10-21,E6609,2025-12-10T15:04:28.710Z


In [0]:
# STOP HERE - THIS IS NOT PART OF EXECUTION. 
# THIS IS FOR DEBUGGING PURPOSES ONLY. 
# Define the SQL query as a Python string
query_zonelog = """
    select d.DomainName, c.DAPClientName, z.name, max(filedate) as max_file_date
    from cdap.ZoneRunLog rl
    join cdap.Domain d on d.DomainId = rl.domainId
    join cdap.Client c on c.ClientKey = rl.clientId
    join cdap.ProcessingZone z on z.zoneId = rl.zoneId
    where lower(c.DAPClientName) LIKE lower('%FirstBank_UHC%')
    group by d.DomainName, c.DAPClientName, z.name
"""

# Use the correct variable in the JDBC read
cdap_zonelog_result_df = (
    spark.read.format("jdbc")
    .option("url", jdbc_url)
    .option("query", query_zonelog)
    .option("user", sql_server_user_name)
    .option("password", sql_server_password)
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver")
    .load()
)

display(cdap_zonelog_result_df)

DomainName,DAPClientName,name,max_file_date
PlanSponsorClaims,FirstBank_UHC,access,20251201
PlanSponsorClaims,FirstBank_UHC,curation,20251201
PlanSponsorClaims,FirstBank_UHC,prep,20251201
PlanSponsorClaims,FirstBank_UHC,structured,20251201
